Goal: find all stations with any relevant data in Oct 2018 - Dec 2023

Order of operations:
- Find all active stations in this time frame
- get dataset info for each that could match 
- check variable names for key words
- save that dataset location and active time range

In [2]:
# pip install --quiet erddapy

Note: you may need to restart the kernel to use updated packages.


In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import urllib.request
import json 
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# help(ERDDAP.get_search_url) 

In [2]:
# load a list of all the ERDDAPs and their urls.
# This is saved as a JSON with the code bc I cant get the direct link to the git site to work.
f = open('erddaps.json')
data_json = pd.read_json(f)
erddap_links = data_json['url'].unique()

erddap_links=erddap_links

print(erddap_links)
# # load in all possible erddap servers straight from site (not working...why??):
# url = 'https://github.com/IrishMarineInstitute/awesome-erddap/blob/df70c66a0784a384f398492d356df46dae4281d4/erddaps.json'
# response = urllib.request.urlopen(url)
# data_json = json.loads(response.read())


(62,)


In [3]:
# try searching for all possibly matching stations for this time (there will be a ton)


min_time = "2018-10-01T00:00:00Z"
max_time = "2024-01-01T00:00:00Z"

kw = {
    "min_time": min_time,
    "max_time": max_time,
}

# save a dataframe of every station that has data in the range
# this is a fast loop that makes the next (very slow) loop faster

# initialize empty lists
allsites = list()
allurl = []
for orgs in erddap_links:
    # print('Searching ' + orgs)
    
    # start making the erddap object for this erddap url 
    e = ERDDAP(
        server=orgs, 
        protocol="tabledap", # Want table data (not a grid map of data) 
        response="csv") #in csv format for pandas

    url = e.get_search_url(search_for='', response="csv", **kw)# search for a match to our time/location
    try:
        temp = pd.read_csv(url)["Dataset ID"].unique() # try to save the data
    except:
        print("----No matching sites exist") # ignore this entry if there is no data
    else:
        print("-Saving matching sites")
        allsites.extend(temp) # save the site name(s)
        for _ in range(len(temp)): allurl.append(orgs) # save a url for each site

# add blank columns for saving the time range and lat/lon range for each match
FF = pd.DataFrame(allsites,columns=['sites'])
FF = FF.assign(url=allurl)
FF["time_coverage_start"] = np.nan
FF["time_coverage_end"] = np.nan
FF["geospatial_lat_min"] = np.nan
FF["geospatial_lat_max"] = np.nan
FF["geospatial_lon_min"] = np.nan
FF["geospatial_lon_max"] = np.nan
FF["check_sum"] = 1
# note: some sites have messy metadata (eg listing lat/lon everywhere) and will show up as a match even when there is no relevant data.
print(len(FF))
del e

-Saving matching sites
-Saving matching sites
-Saving matching sites
----No matching sites exist
----No matching sites exist
-Saving matching sites
-Saving matching sites
-Saving matching sites
-Saving matching sites
-Saving matching sites
-Saving matching sites
----No matching sites exist
-Saving matching sites
-Saving matching sites
-Saving matching sites
-Saving matching sites
----No matching sites exist
-Saving matching sites
-Saving matching sites
-Saving matching sites
----No matching sites exist
-Saving matching sites
-Saving matching sites
----No matching sites exist
----No matching sites exist
-Saving matching sites
-Saving matching sites
----No matching sites exist
-Saving matching sites
-Saving matching sites
----No matching sites exist
-Saving matching sites
-Saving matching sites
-Saving matching sites
-Saving matching sites
----No matching sites exist
-Saving matching sites
-Saving matching sites
-Saving matching sites
-Saving matching sites
-Saving matching sites
----No 

In [109]:
# now search for matching keywords in each erddap site found above (~40k entries)
# I only pull in metadata from ERDDAP (no actual data)
# this cell takes a looooong time (2 h?)
searchterms = r"chl|turb|sedi|par|phot|secchi|ssc"

for jj in range(len(FF)):
    if jj % 100 ==0:
        print(jj)# give a printout every 100 for my sanity
    
    # make the info URL for this site for the full icesat-2 time range
    e = ERDDAP(server=FF['url'][jj], protocol="tabledap", response="csv")
    e.dataset_id = FF['sites'][jj]
    e.constraints = {"time>=": min_time, "time<=": max_time}
    info_url = e.get_info_url()
#     print(info_url)

    # make a dataframe for all the metadata for this station
    # some of these urls are bogus - if fail then fill with NaNs
    try:
        df = pd.read_csv(info_url)# make a dataframe for all the metadata for this station
    except:
        FF.loc[jj,"check_sum"] = np.nan
#         print(str(jj),' does not exist')    
    else:
        # search the Variable Names for relevant measurements and minimum amount of metadata (lat/lon)
        idx = df["Variable Name"].str.contains(searchterms, flags = re.IGNORECASE)
        if any(idx==True)==True:# if there IS a relevant variable        
            e.constraints = {}# find the entire time range for the station and its location
            info_url = e.get_info_url() # just get metadata instead of downloading data(e.get_download_url)
            try:
                df = pd.read_csv(info_url)# make a dataframe for all the metadata for this station
            except:
                FF.loc[jj,"check_sum"] = np.nan
#                 print(str(jj),' does not exist')
            else:

                # sometimes metadata is missing - leave blank if any is empty
                try:
                    FF.loc[jj,"time_coverage_start"] = df.loc[df['Attribute Name']=='time_coverage_start', 'Value'].item()
                except:
                    pass

                try:
                    FF.loc[jj,"time_coverage_end"] = df.loc[df['Attribute Name']=='time_coverage_end', 'Value'].item()
                except:
                    pass

                try:
                    FF.loc[jj,"geospatial_lat_min"] = df.loc[df['Attribute Name']=='geospatial_lat_min', 'Value'].item()
                except:
                    pass

                try:
                    FF.loc[jj,"geospatial_lat_max"] = df.loc[df['Attribute Name']=='geospatial_lat_max', 'Value'].item()
                except:
                    pass

                try:
                    FF.loc[jj,"geospatial_lon_min"] = df.loc[df['Attribute Name']=='geospatial_lon_min', 'Value'].item()
                except:
                    pass

                try:
                    FF.loc[jj,"geospatial_lon_max"] = df.loc[df['Attribute Name']=='geospatial_lon_max', 'Value'].item()
                except:
                    pass

        else: # otherwise ignore this site
#             print(str(jj),' has no relevant data')
            FF.loc[jj,"check_sum"] = np.nan

# remove any entries that failed
FF = FF.dropna(subset="check_sum")  
FF.reset_index(drop=True, inplace=True)
FF = FF.drop(columns="check_sum")
# make any text entries into actual numbers
FF["geospatial_lat_min"] = pd.to_numeric(FF["geospatial_lat_min"])
FF["geospatial_lon_min"] = pd.to_numeric(FF["geospatial_lon_min"])
FF["geospatial_lat_max"] = pd.to_numeric(FF["geospatial_lat_max"])
FF["geospatial_lon_max"] = pd.to_numeric(FF["geospatial_lon_max"])
# save for future reference
FF.to_pickle("all_relevant_stations.pkl")

# also save a csv for curious viewers
FF_save.to_csv('erddap_station_search_results.csv',index=False)
print(len(FF_save))
FF_save

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
# after running that slow code I realized that I didnt save enough information...


# add columns to identify which type of data is present in each dataset
lib = {'buoy','glider','ship','turbid','ssc','photo','chl','radiation'}

FF["buoy"] = False
FF["glider"] = False
FF["ship"] = False
FF["turbid"] = False
FF["ssc"] = False
FF["photo"] = False
FF["chl"] = False
FF['radiation'] = False
FF['bad'] = False

for jj in range(len(FF)):
    if jj % 100 ==0:
        print(jj)# give a printout every 100 for my sanity

    # make the info URL for this site for this time range
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv"
              )
    e.dataset_id = FF['sites'][jj]
    # e.constraints = {"time>=": min_time, "time<=": max_time}
    info_url = e.get_info_url()
    try:
        ds_info = pd.read_csv(info_url)# make a dataframe for all the metadata for this station
    except:
        FF.loc[jj,"bad"] = True
    else:
        for kwd in lib:
            idx = ds_info["Value"].str.contains(kwd, flags = re.IGNORECASE)
            if any(idx==True)==True:
                FF.loc[jj,kwd] = True

FF = FF[FF.bad == False] 
FF.reset_index(drop=True, inplace=True)
FF = FF.drop(columns="bad")
FF.to_pickle("labeled_relevant_stations.pkl")

# now we can look at how many stations are available for each data type
print(len(FF[(FF.buoy == True) & (FF.radiation == True)]))
FF[(FF.ship == True) & (FF.radiation == True)]